In [1]:
import pandas as pd
import numpy as np
import re

import plotly.express as px
import datetime
from Project.Database import Db
from Project._05InferKnowledgeOfRules.infer_rules_functions import SE_time_df, json_to_dataframe
from Project._07OptimiseConsumption.optimisation_problem import NZERTF_optimiser, hourly_house_df
from Project._07OptimiseConsumption.emission_reduction import emission_reduction

In [ ]:
NZERTF_optimisation, NZERTF_emission = emission_reduction(year=2)

In [ ]:
NZERTF_meta = Db.load_data(hourly=False, meta=True, year=2, with_redundancy=True, consumption=False)

In [ ]:
def create_intervals(start_, end_, start_index_, end_index_, start_frac_, end_frac_):
    if start_index_ != end_index_:
        diff_start_ = start_ + pd.DateOffset(hours=1, minutes=0) - start_
        minutes_start_ = diff_start_.seconds / 60 * start_frac_
        start_ = start_ + pd.DateOffset(hours=1, minutes=0) - pd.DateOffset(hours=0, minutes=minutes_start_)

        diff_end_ = end_ - (end_ - pd.DateOffset(hours=1, minutes=0))
        minutes_end_ = diff_end_.seconds / 60 * end_frac_
        end_ = end_ + pd.DateOffset(hours=0, minutes=minutes_end_)
    else:
        diff_start_ = start_ + pd.DateOffset(hours=1, minutes=0) - start_
        minutes_start_ = diff_start_.seconds / 60 * start_frac_
        start_ = start_ + pd.DateOffset(hours=1, minutes=0) - pd.DateOffset(hours=0, minutes=minutes_start_)

        end_ = end_ + pd.DateOffset(hours=1, minutes=0)

    return start_, end_

In [ ]:
appliance_job_list = []
for df in [NZERTF_optimisation['m.a.u.o'], NZERTF_optimisation['m.a.o']]:
    #Extract appliances from the meta data
    appliance_condition = [x for x in NZERTF_optimisation['m.a.o'].columns.tolist() if "Status" in x and 'Emission' not in x and 'EventProportion' not in x]
    appliance_location = pd.DataFrame(NZERTF_meta.loc[appliance_condition, "Measurement_Location"].sort_values(ascending=True))
    #Combine minute data with the extracted appliances into a dataframe
    minute_appliances_status = df[["Timestamp"] + appliance_location.index.tolist()].copy()
    for appliance, appliance_row in appliance_location.iterrows():
        name = " ".join(re.findall('[A-Z][^A-Z]*', ''.join(''.join(''.join(appliance.split('PowerTotal')).split('Appliance')).split('PlugLoad')).split("Load_Status")[-1]))
        appliance_switch = minute_appliances_status[(np.where(minute_appliances_status[appliance] > 0, 1, 0) != np.where(minute_appliances_status[appliance].shift(1)>0,1,0))][["Timestamp", appliance]][1:]
        appliance_switch.reset_index(inplace=True, drop=True)
        event_length = df.loc[lambda self: self[appliance + 'EventProportion'] > 0, appliance].div(df.loc[lambda self: self[appliance + 'EventProportion'] > 0, appliance + 'EventProportion']).mean().round(4)
        for index, row in appliance_switch.iterrows():
            if row[appliance]:
                try:
                    start = row["Timestamp"]
                    end = appliance_switch.loc[index + 1, "Timestamp"] - pd.DateOffset(hours=1, minutes=0)
                    start_index = df.loc[lambda self: self['Timestamp']==start].index.tolist()[0]
                    end_index = df.loc[lambda self: self['Timestamp']==end].index.tolist()[0]
                    start_frac = df.loc[lambda self: self['Timestamp']==start,appliance].values[0]
                    end_frac = df.loc[lambda self: self['Timestamp']==end,appliance].values[0]

                    events_indexes = df.loc[lambda self: self['Timestamp'].between(start, end)].index.tolist()
                    events_length = df.loc[events_indexes, appliance].sum()
                    event_proportion = df.loc[events_indexes, appliance + 'EventProportion'].sum()
                    event_count = int(max(round(event_proportion, 0), 1))
                    event_length = events_length / event_count

                    start_row_frac = 1
                    temp_start = start
                    temp_start_index = start_index
                    temp_start_frac = start_frac

                    temp_end = start + pd.DateOffset(hours=int(event_length), minutes=0)
                    temp_end_frac = df.loc[lambda self: self['Timestamp']==temp_end,appliance].values[0]

                    for number in range(event_count):
                        temp_end_index = df.loc[start_index: end_index+1].loc[lambda self: self[appliance].cumsum().div(event_length).subtract(number) >= 1, appliance].index[0]
                        temp_end = df.loc[temp_end_index, 'Timestamp']
                        end_row_frac = df.loc[start_index: temp_end_index, appliance].cumsum().div(event_length).subtract(number).round(4).values[-1]
                        emission = df.loc[temp_start_index: temp_end_index, appliance + 'Emission'].round(2)
                        emission[temp_start_index] = round(emission[temp_start_index] * start_row_frac, 2)

                        if end_row_frac > 1:
                            end_row_frac = round(2 - end_row_frac, 4)
                            temp_end_frac = df.loc[temp_end_index, appliance] * end_row_frac
                            emission[temp_end_index] = round(emission[temp_end_index] * end_row_frac, 2)
                            next_start_row_frac = round(1 - end_row_frac, 4)
                            next_start_index = temp_end_index.copy()
                            next_start_frac = round(df.loc[temp_end_index, appliance] * next_start_row_frac, 4)
                        else:
                            temp_end_frac = df.loc[temp_end_index, appliance]
                            next_start_row_frac = 1
                            next_start_index = temp_end_index + 1
                            next_start_frac = df.loc[temp_end_index + 1, appliance]

                        event_start, event_end = create_intervals(temp_start, temp_end, temp_start_index, temp_end_index, temp_start_frac, temp_end_frac)

                        if NZERTF_optimisation['m.a.u.o']['Emission'].sum() == df['Emission'].sum():
                            appliance_job_list.append({"appliance": name+' before', "start": event_start,
                                                           "end": event_end,
                                                           "location": appliance_row["Measurement_Location"],
                                                       'CO2 Emission (kg)': round(emission.sum(), 2)})
                        else:
                            appliance_job_list.append({"appliance": name+' after',
                                                       "start": event_start,
                                                       "end": event_end,
                                                       "location": appliance_row["Measurement_Location"],
                                                       'CO2 Emission (kg)': round(emission.sum(), 2)})

                        start_row_frac = next_start_row_frac
                        temp_start_index = next_start_index
                        temp_start = df.loc[temp_start_index, 'Timestamp']
                        temp_start_frac = next_start_frac

                except:
                    continue

In [ ]:
%%time
fig = px.timeline(appliance_job_list, x_start="start", x_end="end", y="appliance", color='CO2 Emission (kg)', text='CO2 Emission (kg)', category_orders={"appliance": [
    'Dishwasher after',
    'Dishwasher before',
    'Clothes Washer after',
    'Clothes Washer before',
    'Dryer after',
    'Dryer before',
    'Iron after',
    'Iron before',
    'Vacuum after',
    'Vacuum before']})
fig.update_layout(template='plotly')